# Notebook 6 - Label Thresholds and Cleaning
* The labeled dataframe had probabilities of a text being classified as that label. Thresholds need to be picked rather than blindly sticking with .5 due to weight of curse words being different in the gaming community.
* Understanding and changing the most common (correctly generalizable) false positives

In [115]:
import numpy as np
import pandas as pd

In [116]:
dota = pd.read_csv('dota_with_labels.csv').drop('Unnamed: 0', axis=1)
dota.head()

,match,slot,time,text,toxic,obscene,threat,insult,identity_hate
0,0,9,1808.40822,100%,0.030836,0.009912,0.000770,0.007847,0.001074
1,2,6,1238.43090,carry,0.015985,0.003671,0.000536,0.004548,0.002409
2,2,0,1248.42850,yes dog,0.052213,0.005909,0.001787,0.024825,0.004618
3,2,0,1559.05260,yeah,0.026854,0.010703,0.000929,0.008751,0.006737
4,2,0,1563.18490,fast and furious,0.220139,0.103201,0.002675,0.034582,0.004633


# Selecting Thresholds
* Finding thresholds to determine labels (manually bc gaming context)

**More lines were observed than the notebook shows, but the tables were cut short to maintain cleanliness of the notebook.**

## Toxic

In [117]:
dota[dota['toxic'] > .6].sort_values('toxic').head()

,match,slot,time,text,toxic,obscene,threat,insult,identity_hate
5764532,832407,3,345.58230,KILLED M?,0.600000,0.025863,0.047310,0.039227,0.005516
2097991,302455,4,4.33230,u are all on twitch,0.600002,0.176116,0.002425,0.333036,0.012875
2304691,332009,6,-37.39086,Dont go all out guys :(,0.600007,0.107021,0.007464,0.156044,0.048475
1915602,276112,5,729.58856,dusa is mid kill her plz,0.600016,0.064399,0.136882,0.034257,0.010585
5541106,799689,6,1892.25540,give me rampage pussy,0.600018,0.649396,0.000595,0.055104,0.003376


* .6 seems too low

In [118]:
dota[dota['toxic'] > .7].sort_values('toxic').head()

,match,slot,time,text,toxic,obscene,threat,insult,identity_hate
4334581,626849,4,26.53900,riki those pussy,0.700001,0.583243,0.000637,0.057004,0.007796
5262600,759839,3,-277.92723,wtf is ur day,0.700008,0.263107,0.001921,0.061213,0.011240
5108145,737773,8,685.31690,enough 3 dog to kill me,0.700010,0.023357,0.206505,0.012125,0.005862
3692858,533457,7,1439.08192,base haha fck you,0.700013,0.495393,0.003047,0.246655,0.007405
761798,110009,3,2142.44280,no just our tiny is very dumb,0.700014,0.108703,0.001025,0.389209,0.011677


* .7 is definitely better than .6, but still seems to be a little low if these toxicity labels are going to result in punishment

In [119]:
dota[dota['toxic'] > .75].sort_values('toxic').head()

,match,slot,time,text,toxic,obscene,threat,insult,identity_hate
5751685,830672,8,1901.75233,DAT LUCK THO,0.750000,0.266125,0.001818,0.045170,0.006250
589961,85177,6,640.01043,i want jug to kill me,0.750001,0.053834,0.428790,0.014289,0.008502
4768170,689083,8,933.27210,'FEED UR FACE U skill and feed?,0.750001,0.278987,0.056804,0.278612,0.016245
4231780,611956,7,2661.52070,puck my mvp,0.750011,0.602694,0.001326,0.046546,0.004904
6632962,959081,7,1884.34864,I AM A DELUDED 4K YOU PEASANT,0.750016,0.069716,0.007526,0.091951,0.005311


In [120]:
dota[dota['toxic'] > .8].sort_values('toxic').head(15)

,match,slot,time,text,toxic,obscene,threat,insult,identity_hate
6396687,925051,4,767.04600,u crazy guys XD,0.800004,0.118128,0.001368,0.399497,0.029799
2319192,334214,8,1281.09017,ukraine is russia's niggers,0.800005,0.357961,0.001018,0.344439,0.920623
6527867,944019,3,3475.98795,WHUT THE HECK,0.800010,0.066565,0.001393,0.018210,0.004101
589980,85177,6,734.18743,let your jug get fat,0.800012,0.190509,0.009608,0.475129,0.042192
5190567,749542,1,546.07585,pa low jungle go kill,0.800013,0.056746,0.099258,0.070570,0.008705
6361688,919944,6,1253.92038,U LOOE,0.800016,0.080739,0.006094,0.314560,0.009477
1185433,171040,4,2091.44440,damn this is a carry,0.800021,0.638700,0.000309,0.078034,0.016090
823021,118747,4,-36.02455,i hope u eat pussy right now,0.800029,0.501663,0.027883,0.124050,0.008171
5374918,775880,9,804.49382,widra wtf,0.800032,0.427039,0.002062,0.014070,0.003627
2015802,290644,3,2209.76042,i play with the most retarded ck,0.800038,0.052116,0.001406,0.114251,0.017709


Looking at only the 'toxic' label presents many rows that would be considered false positives in the gaming world, but the 'toxic' label can be helpful by flagging texts and having a moderator manually judge these messages.

On the other hand, the other labels are more telling, such as obscene, threat, insult, and identity_hate.
* Some false positives are due to cuss words ("windra wtf") and slang ("sick ganks", the word "kill"), but that is not necessarily toxic. 'obscene' captures the use of cuss words.
* **Threat, insult, and identity_hate are more important because they are signs of attacking and most likely offending other players, which seems like the basis of punishment.**
    * Because these are more important, 'toxic' will keep a high threshold to account for the gaming standards.

In [121]:
dota[dota['toxic'] > .825].sort_values('toxic').head()

,match,slot,time,text,toxic,obscene,threat,insult,identity_hate
3435845,496181,4,1925.36920,IT FACKING TICKELS,0.825001,0.504502,0.001948,0.073314,0.005178
4763079,688373,9,-29.39282,oh damn here we go,0.825002,0.731629,0.001365,0.151228,0.023645
1094299,158281,2,1231.26602,yes shame on u cancer picker,0.825005,0.150593,0.001587,0.231182,0.010012
4665324,674775,5,2167.14230,HE ISSS SUCH A NOOOB,0.825012,0.189292,0.002601,0.148223,0.013254
5440487,785260,2,455.72210,WTF THIS GUY,0.825013,0.238241,0.001197,0.081561,0.009382


In [122]:
dota[dota['toxic'] > .85].sort_values('toxic').head()

,match,slot,time,text,toxic,obscene,threat,insult,identity_hate
2892138,416772,3,762.24720,complete morons,0.850000,0.061771,0.000427,0.536521,0.001863
4505299,651683,8,2411.02803,HAHAHAH YEAH WORST LC EVER,0.850007,0.034418,0.004067,0.084519,0.012296
4024083,581386,2,2559.47367,we don't need no fancy ass carries here,0.850008,0.642435,0.001135,0.139274,0.014624
11614,1746,6,1903.20828,give u kill cant carry sf,0.850010,0.133581,0.128872,0.071135,0.033548
2618670,377341,7,932.43900,Retard picked pl,0.850013,0.134602,0.001147,0.588902,0.006933


> **A threshold of .85 will be used for 'toxic'.**

## Obscene

In [123]:
dota[dota['obscene'] > .5].sort_values('obscene').head()

,match,slot,time,text,toxic,obscene,threat,insult,identity_hate
3318497,478656,0,2175.70053,hahaha wtf,0.425208,0.500003,0.001740,0.015846,0.004676
5201274,751133,2,1963.96698,lost against IDIOT,0.998590,0.500006,0.000717,0.976934,0.013399
6012331,869221,7,365.27750,2 shits again,0.855081,0.500038,0.002051,0.190669,0.019818
4884629,705914,9,1522.12832,mothersucker bs ever reported,0.863062,0.500044,0.001246,0.322537,0.009956
2904811,418670,2,1243.36306,{DFWA{F{,0.775610,0.500045,0.001087,0.056083,0.006425


In [124]:
dota[dota['obscene'] > .55].sort_values('obscene').head()

,match,slot,time,text,toxic,obscene,threat,insult,identity_hate
4355058,629831,5,2063.38310,push idiots,0.997857,0.550014,0.001137,0.991731,0.008007
4053402,585732,5,2574.44695,push idiots,0.997857,0.550014,0.001137,0.991731,0.008007
5496426,793197,6,3751.47485,push idiots,0.997857,0.550014,0.001137,0.991731,0.008007
1364505,196963,3,1644.93283,push idiots,0.997857,0.550014,0.001137,0.991731,0.008007
1388495,200473,8,2036.49210,push idiots,0.997857,0.550014,0.001137,0.991731,0.008007


In [125]:
dota[dota['obscene'] > .6].sort_values('obscene').head(10)

,match,slot,time,text,toxic,obscene,threat,insult,identity_hate
1954217,281650,1,1674.99100,and zues can't roam or do shit for the team,0.845077,0.600001,0.001611,0.042168,0.006452
3787947,547292,6,-74.15532,you're so damn pretty sharla,0.873963,0.600014,0.000964,0.113361,0.014671
4785704,691609,0,2418.44150,sf commend asshhole,0.648718,0.600021,0.001408,0.197446,0.004357
2952251,425671,9,1615.37226,Nice grammar idiot,0.995223,0.600029,0.000753,0.957245,0.005756
1145321,165559,7,569.69424,learn your damn vocabulary,0.727435,0.600032,0.002988,0.046795,0.013420
5243648,757078,5,1237.76448,ARC WARDEN STOP SPAMMING CHAT that you are pro...,0.941072,0.600033,0.007347,0.395726,0.039131
5731988,827769,9,1986.64747,pl miss me w that illusion shit,0.969177,0.600047,0.001636,0.044664,0.004520
748401,107957,1,1619.43790,rubrick waht do u feel about this puck 1 hitti...,0.824507,0.600054,0.001058,0.132122,0.010219
4946773,714894,4,1092.16664,get cleaned up bitches,0.673161,0.600056,0.001554,0.428200,0.005007
2735643,394259,2,-228.96398,was just being stupid.. more of you then us..,0.993259,0.600060,0.006811,0.873516,0.016503


> **A threshold of .6 will be used for 'obscene'.**

## Threat

In [126]:
dota[dota['threat'] > .6].sort_values('threat').head()

,match,slot,time,text,toxic,obscene,threat,insult,identity_hate
3950978,570831,0,1886.18598,focue more and die all,0.806236,0.070112,0.600087,0.107612,0.006388
2202100,317073,7,952.30419,and i can kill you alone lol,0.876673,0.067737,0.600097,0.088414,0.009128
2299618,331304,8,1198.43060,i will help you with my ulti if someone gonna ...,0.485872,0.091397,0.600217,0.040261,0.006315
3844716,555486,7,3299.43542,you noob you cant kill me,0.972985,0.209186,0.600304,0.368070,0.018838
3869469,559085,1,758.44820,i hate to kill,0.981512,0.035338,0.600325,0.024648,0.064081


* Much of 'threat' seems to be based on the word 'kill'. A higher threshold will be used to account for this.

In [127]:
dota[dota['threat'] > .75].sort_values('threat').head()

,match,slot,time,text,toxic,obscene,threat,insult,identity_hate
1619077,233632,7,2146.24156,watch and die,0.663525,0.082348,0.750032,0.069855,0.005692
3194538,460596,9,731.28812,you let your mid die,0.967710,0.227547,0.750032,0.436722,0.011138
6053674,875165,6,1179.21202,WE CANT KILL,0.939783,0.069282,0.750139,0.030926,0.047423
5372917,775590,4,1764.72215,"well, then i kill u",0.816150,0.084771,0.750274,0.048510,0.008452
2515096,362294,2,-51.05420,i wanna kill ymself,0.734850,0.091893,0.750290,0.009928,0.012222


In [128]:
dota[dota['threat'] > .8].sort_values('threat').head()

,match,slot,time,text,toxic,obscene,threat,insult,identity_hate
315250,45489,3,683.16654,can you kill,0.891074,0.110159,0.800026,0.075442,0.034312
6019773,870284,4,1751.75127,can you kill,0.891074,0.110159,0.800026,0.075442,0.034312
2531910,364599,9,2545.09870,or I will rape your family,0.622752,0.117301,0.800139,0.112274,0.033400
3787357,547197,9,1123.05915,no kill for you,0.853140,0.066856,0.800162,0.044068,0.008035
6696387,968113,2,2282.01382,no kill for you,0.853140,0.066856,0.800162,0.044068,0.008035


In [129]:
tmp = dota[~dota['text'].str.lower().str.contains('kill')]
tmp[tmp['threat'] > .8].sort_values('threat').head()

,match,slot,time,text,toxic,obscene,threat,insult,identity_hate
2531910,364599,9,2545.09870,or I will rape your family,0.622752,0.117301,0.800139,0.112274,0.033400
4372833,632310,1,1958.16568,mafia we will find you and die....,0.486659,0.040416,0.800545,0.057846,0.012605
2419854,348482,8,305.91080,mess with me you will die,0.857785,0.070733,0.800748,0.125511,0.009185
4618970,667941,2,159.82763,you die and blame?,0.872278,0.084790,0.800754,0.168584,0.018673
4679000,676717,7,694.99700,i hope your mom ll die from cancer,0.901339,0.058818,0.800759,0.189920,0.015051


* Removing 'kill' looks better for labeling

In [130]:
tmp[tmp['threat'] > .75].sort_values('threat').head(15)

,match,slot,time,text,toxic,obscene,threat,insult,identity_hate
1619077,233632,7,2146.24156,watch and die,0.663525,0.082348,0.750032,0.069855,0.005692
3194538,460596,9,731.28812,you let your mid die,0.967710,0.227547,0.750032,0.436722,0.011138
588533,84954,4,1798.80848,go die with your mom,0.989150,0.305783,0.750567,0.702128,0.039734
4629724,669565,6,1121.82604,AFTER I DIE,0.864288,0.096313,0.750753,0.046786,0.004341
521725,75571,2,953.73378,i hope you die tomorrow sniper,0.681196,0.030534,0.750909,0.198661,0.014584
4438913,641904,4,-42.95618,THEN WE DIE,0.923123,0.063921,0.750941,0.059977,0.008832
3037740,438014,3,2093.24715,then hope u die in real life,0.815656,0.033879,0.750999,0.101944,0.006073
2153757,310144,1,975.39520,i will rape you hard,0.786803,0.245460,0.751284,0.119816,0.030441
3951832,570968,5,690.26483,you death?,0.385944,0.034678,0.751845,0.083208,0.025902
6044465,873929,2,756.01536,u will die with 1 hit,0.751364,0.083372,0.751858,0.059061,0.010255


* The word 'die' is a big issue in threats because it is a term to use in-game but there are also death threats, as shown by the third row's text, "go die with your mom".
* Overall, there are way too many messages where 'die' is used casually in game and they are almost always directed at other players, so it is hard to separate.
* A threshold of .75 still contains threatening messages, so a higher one will not be used.

> **A threshold of .75 will be used for 'threat'.**

## Insult

In [131]:
dota[dota['insult'] > .5].sort_values('insult').head(10)

,match,slot,time,text,toxic,obscene,threat,insult,identity_hate
6430095,929880,5,1674.257830,I understand you are most likely a nigger,0.879640,0.276940,0.004810,0.500004,0.682262
2029608,292606,8,1059.874540,fuck mother of mirana,0.999786,0.999440,0.001582,0.500011,0.010050
630653,91005,3,423.063390,lmao he suck anyway <3,0.984696,0.910169,0.000685,0.500016,0.009525
294786,42687,1,2012.605000,REPORT THIS PIECE OF SHIT,0.999622,0.982947,0.002303,0.500017,0.017524
2188847,315102,4,1264.391260,REPORT THIS PIECE OF SHIT,0.999622,0.982947,0.002303,0.500017,0.017524
5576476,804825,2,2453.981800,REPORT THIS PIECE OF SHIT,0.999622,0.982947,0.002303,0.500017,0.017524
5811611,839419,6,-259.705165,your lc is retarded :D,0.779766,0.030041,0.003145,0.500023,0.023299
6128703,886030,5,2586.507260,this retard sniper,0.668619,0.026804,0.001048,0.500025,0.005981
4524437,654368,5,962.265030,this weaver so fucking good,0.998372,0.999385,0.005321,0.500026,0.047911
1690843,243999,8,1642.065720,"nah u are so dumb, u gank me 10 times and get ...",0.905667,0.194766,0.002933,0.500042,0.027921


> **A threshold of .5 will be used for 'insult'.**

## Identity Hate

In [132]:
dota[dota['identity_hate'] > .5].sort_values('identity_hate').head(10)

,match,slot,time,text,toxic,obscene,threat,insult,identity_hate
974444,140711,7,1269.68409,if i were a nigga in russia i would also be sad,0.465319,0.082729,0.008861,0.083833,0.500106
5666813,817921,3,1491.05729,easy viper gay,0.758477,0.030547,0.001448,0.153456,0.500545
3032642,437297,9,1105.02254,i have fuckin niggas in my team,0.996629,0.996224,0.004723,0.798399,0.500580
4348409,628830,5,1667.91821,run russian faggot run,0.941849,0.531934,0.002164,0.681365,0.500582
6371264,921438,9,613.75505,fucking russian faggets,0.999606,0.999780,0.004836,0.925085,0.500582
5214739,752990,5,2969.41570,gg nigga first time win haha,0.448223,0.174927,0.002382,0.254226,0.500601
6164973,890963,5,1240.99965,least i aint gay,0.866812,0.047294,0.003514,0.055590,0.500652
6689018,967163,3,2021.35675,retarded anime faggot,0.998989,0.816742,0.002964,0.986417,0.500915
23696,3508,3,1214.99582,why me all time you gay ?,0.931859,0.098045,0.019910,0.263429,0.500990
839254,121036,7,-67.78344,smd faggot,0.991243,0.925746,0.003049,0.933338,0.501016


> **A threshold of .5 will be used for 'identity_hate'.**

## Thresholds
* Toxic: .85
* Obscene: .6
* Threat: .75
* Insult: .5
* Identity_hate: .5

In [133]:
dota['toxic'] = (dota['toxic'] > .85).replace(True, 1).apply(int)
dota['obscene'] = (dota['obscene'] > .6).replace(True, 1).apply(int)
dota['threat'] = (dota['threat'] > .75).replace(True, 1).apply(int)
dota['insult'] = (dota['insult'] > .5).replace(True, 1).apply(int)
dota['identity_hate'] = (dota['identity_hate'] > .5).replace(True, 1).apply(int)
dota

,match,slot,time,text,toxic,obscene,threat,insult,identity_hate
0,0,9,1808.40822,100%,0,0,0,0,0
1,2,6,1238.43090,carry,0,0,0,0,0
2,2,0,1248.42850,yes dog,0,0,0,0,0
3,2,0,1559.05260,yeah,0,0,0,0,0
4,2,0,1563.18490,fast and furious,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
6914908,999998,8,917.21927,damn you!!!!,1,1,0,0,0
6914909,999998,6,1709.49237,baited,0,0,0,0,0
6914910,999998,7,1765.54537,lmao,0,0,0,0,0
6914911,999999,0,974.04976,sec please,0,0,0,0,0


In [134]:
for col in dota.columns[4:]:
    print(dota[col].sum(), col, "messages out of", len(dota[col]), 
          "({}%)".format(round(dota[col].sum()/len(dota[col]) * 100, 2)))

453624 toxic messages out of 6914913 (6.56%)
399490 obscene messages out of 6914913 (5.78%)
5940 threat messages out of 6914913 (0.09%)
271771 insult messages out of 6914913 (3.93%)
14358 identity_hate messages out of 6914913 (0.21%)


In [135]:
# # Save for future analysis
# dota.to_csv('dota_with_labels_2.csv', index=False)

# Deeper Label Cleaning

In [232]:
dota = pd.read_csv('dota_with_labels_2.csv')
dota.head()

,match,slot,time,text,toxic,obscene,threat,insult,identity_hate
0,0,9,1808.40822,100%,0,0,0,0,0
1,2,6,1238.43090,carry,0,0,0,0,0
2,2,0,1248.42850,yes dog,0,0,0,0,0
3,2,0,1559.05260,yeah,0,0,0,0,0
4,2,0,1563.18490,fast and furious,0,0,0,0,0


In [233]:
dota.shape

(6914913, 9)

In [234]:
# Reference of counts before cleaning
for col in dota.columns[4:]:
    print(dota[col].sum(), col, "messages out of", len(dota[col]), 
          "({}%)".format(round(dota[col].sum()/len(dota[col]) * 100, 2)))

453624 toxic messages out of 6914913 (6.56%)
399490 obscene messages out of 6914913 (5.78%)
5940 threat messages out of 6914913 (0.09%)
271771 insult messages out of 6914913 (3.93%)
14358 identity_hate messages out of 6914913 (0.21%)


In [235]:
# Messages that contain all types of harassment
dota[(dota['threat']==1) & (dota['insult']==1) & (dota['identity_hate']==1)]

,match,slot,time,text,toxic,obscene,threat,insult,identity_hate
4277216,618580,2,2948.58118,I HOPE YOU ALL FUCKING GAY RUSSIAN SUCKER DIE ...,1,1,1,1,1
4304067,622513,5,-54.48670,ok i will rape you fucking faggot,1,1,1,1,1
4610883,666865,3,947.70190,HOPE U DIE TO EBOLA U FUCKING FAGGOTS,1,1,1,1,1
5114798,738771,0,1325.18606,i kill you nigga,1,1,1,1,1
5452593,786925,2,394.17045,techies you are a mother fucking bitch and i h...,1,1,1,1,1


In [236]:
# Messages that contain at least one type of harassment
offensive = dota[(dota['toxic']==1) | (dota['threat']==1) | (dota['insult']==1) | (dota['identity_hate']==1)]
offensive[offensive['text']=='wtf'].head()

,match,slot,time,text,toxic,obscene,threat,insult,identity_hate
10,4,5,37.59082,wtf,1,1,0,0,0
41,9,9,364.87762,wtf,1,1,0,0,0
109,19,3,1286.71916,wtf,1,1,0,0,0
177,32,5,1363.73367,wtf,1,1,0,0,0
192,33,5,441.14938,wtf,1,1,0,0,0


## Cleaning 'Toxic'

In [237]:
dota[dota['text'].str.lower()=='wtf'].shape

(56980, 9)

* 56,980 messages are considered toxic by only saying 'wtf'. This will be changed because, despite containing a swear word, the phrase expresses surprise or disbelief (ie. "wtf are you doing?")

In [239]:
# Changing 'toxic' to 0 for text with only 'wtf'
dota.loc[dota['text'].str.lower() =='wtf', 'toxic'] = 0
dota[dota['text']=='wtf'].head()

,match,slot,time,text,toxic,obscene,threat,insult,identity_hate
10,4,5,37.59082,wtf,0,1,0,0,0
41,9,9,364.87762,wtf,0,1,0,0,0
109,19,3,1286.71916,wtf,0,1,0,0,0
177,32,5,1363.73367,wtf,0,1,0,0,0
192,33,5,441.14938,wtf,0,1,0,0,0


In [240]:
offensive = dota[(dota['toxic']==1) | (dota['threat']==1) | (dota['insult']==1) | (dota['identity_hate']==1)]
offensive.head()

,match,slot,time,text,toxic,obscene,threat,insult,identity_hate
16,6,0,241.44110,so ya mama likes dick ehh?,1,1,0,0,0
18,6,1,241.44110,fucking reported axe,1,1,0,0,0
26,6,1,874.78640,STUPIDD PIUDGE,1,0,0,0,0
27,6,1,876.45270,STUPID!,1,1,0,1,0
65,9,3,2043.83972,fuckING,1,1,0,1,0


In [241]:
# Looking at the text with shortest length so see classification
tmp = offensive.copy()
tmp['text_len'] = tmp['text'].apply(len)
tmp.sort_values('text_len').head()

,match,slot,time,text,toxic,obscene,threat,insult,identity_hate,text_len
3665841,529561,4,-35.52465,F,1,1,0,0,0,1
2677864,385842,2,475.85050,F,1,1,0,0,0,1
3996137,577428,3,1668.12600,F,1,1,0,0,0,1
6317119,913470,0,977.44927,F,1,1,0,0,0,1
397976,57540,7,1721.61292,F,1,1,0,0,0,1


* Single letters are being identified as toxic. This will be changed.

In [242]:
# Changing class for 1-2 letter texts
tmp[tmp['text_len'] <= 2]

,match,slot,time,text,toxic,obscene,threat,insult,identity_hate,text_len
4119,596,4,1245.66250,Ck,1,1,0,0,0,2
9823,1495,8,1915.19678,ck,1,1,0,0,0,2
10402,1575,5,3712.94130,ck,1,1,0,0,0,2
18980,2875,0,2144.62983,F,1,1,0,0,0,1
19381,2934,6,2866.22345,ck,1,1,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...
6892082,996671,0,2876.51216,ck,1,1,0,0,0,2
6892421,996723,5,893.04860,ck,1,1,0,0,0,2
6904828,998543,7,1948.49370,ck,1,1,0,0,0,2
6907471,998911,1,1314.34574,F,1,1,0,0,0,1


* Looking up 'ck' for Dota, it is a hero named Chaos Knight.

In [243]:
# Looking at short text that aren't 'F' or 'ck'
short_toxic = tmp[tmp['text_len'] <= 2][(tmp['text'].str.lower() != 'f') & (tmp['text'].str.lower() != 'ck')]
short_toxic.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,match,slot,time,text,toxic,obscene,threat,insult,identity_hate,text_len
20344,3065,8,1187.87648,*U,1,0,0,0,0,2
25751,3723,0,-4.16565,SH,1,0,0,0,0,2
43361,6191,8,2798.04778,Fu,1,1,0,1,0,2
74806,10677,7,760.88090,Ű,1,0,0,0,0,1
77056,10987,4,1446.08026,Fu,1,1,0,1,0,2


In [244]:
short_toxic[short_toxic['text'].str.lower() != 'fu'].groupby('text').size()

text
 f     2
'F     2
*U     1
:U    35
F     14
F:     3
F=     2
F     1
SH    29
U&     2
U'     1
|F     1
£F     1
Ű      7
dtype: int64

* Besides 'Fu', texts that are 1-2 characters long shouldn't be considered toxic.

In [245]:
dota[dota['text'].str.lower() == 'fu'].shape

(128, 9)

* There are only 128 rows that contain 'Fu' out of around 7 million rows. 'Fu' isn't a ban-able offense and will be considered not toxic for this dataset, but it will still be considered obscene and an insult. 
* All rows with 1-2 characters only will be considered not toxic.

In [246]:
# Making text with len <= 2 not toxic
dota_copy = dota.copy()
dota_copy['text_len'] = dota_copy['text'].apply(len)
dota_copy.loc[dota_copy['text_len'] <= 2, 'toxic'] = 0
dota['toxic'] = dota_copy['toxic']
dota.head()

,match,slot,time,text,toxic,obscene,threat,insult,identity_hate
0,0,9,1808.40822,100%,0,0,0,0,0
1,2,6,1238.43090,carry,0,0,0,0,0
2,2,0,1248.42850,yes dog,0,0,0,0,0
3,2,0,1559.05260,yeah,0,0,0,0,0
4,2,0,1563.18490,fast and furious,0,0,0,0,0


In [247]:
# Top 20 most common offensive texts
offensive = dota[(dota['toxic']==1) | (dota['threat']==1) | (dota['insult']==1) | (dota['identity_hate']==1)]
offensive[offensive['toxic']==1].groupby('text').size().sort_values(ascending=False).head(20)

text
fuck you                         4494
shit                             4300
bitch                            3699
fuck off                         3679
idiots                           2043
shut up                          1800
pussy                            1756
stupid                           1632
fucking                          1614
wtf                              1126
holy shit                        1101
noob jug idiot dog ugly nigga    1052
BITCH                             927
FUCKING                           923
moron                             919
loser                             787
WTF?                              784
suck my dick                      777
kill him                          751
fucking noob                      741
dtype: int64

* There is still "wtf " (with a space) and "WTF?" with a question mark. These will be removed like before.
* "Shit" alone is often "used as an emphatic exclamation roughly meaning 'oh no!'." (https://www.vocabulary.com/dictionary/shit) "Fuck" alone is used the same way. These will also be considered not toxic.
* "Holy shit" expresses surprise or shock and will be removed (not targeting other players).
* "Kill him" will be removed because it is a core part of the gameplay and not meant offensively.

In [248]:
dota.loc[dota['text'].str.lower()==("wtf "), 'toxic'] = 0
dota.loc[dota['text'].str.lower()==("wtf?"), 'toxic'] = 0
dota.loc[dota['text'].str.lower()==("what the fuck"), 'toxic'] = 0
dota.loc[dota['text'].str.lower()==("what the fuck?"), 'toxic'] = 0
dota.loc[dota['text'].str.lower()==("shit"), 'toxic'] = 0
dota.loc[dota['text'].str.lower()==("fuck"), 'toxic'] = 0
dota.loc[dota['text'].str.lower()==("holy shit"), 'toxic'] = 0
dota.loc[dota['text'].str.lower()==("kill him"), 'toxic'] = 0

In [249]:
# The top 15-20 most common offensive texts (updated)
offensive = dota[(dota['toxic']==1) | (dota['threat']==1) | (dota['insult']==1) | (dota['identity_hate']==1)]
offensive[offensive['toxic']==1].groupby('text').size().sort_values(ascending=False).head(20).tail()

text
bitches          695
asshole          689
oh shit          664
the fuck         656
piece of shit    565
dtype: int64

* "oh shit" and "the fuck" will be re-classified for the same reasons as before for "shit" and "wtf".

In [250]:
dota.loc[dota['text'].str.lower()==("oh shit"), 'toxic'] = 0
dota.loc[dota['text'].str.lower()==("the fuck"), 'toxic'] = 0

In [251]:
# The top 15-20 most common offensive texts (updated again)
offensive = dota[(dota['toxic']==1) | (dota['threat']==1) | (dota['insult']==1) | (dota['identity_hate']==1)]
offensive[offensive['toxic']==1].groupby('text').size().sort_values(ascending=False).head(20).tail()

text
bitches           695
asshole           689
piece of shit     565
fuck this game    560
mother fucker     545
dtype: int64

* "Fuck this game" will not be considered toxic because it's not towards other players and players' opinions of the game should not be silenced.

In [252]:
dota.loc[dota['text'].str.lower()==("fuck this game"), 'toxic'] = 0

# An updated look at the top 20 most common toxic texts after cleaning
offensive = dota[(dota['toxic']==1) | (dota['threat']==1) | (dota['insult']==1) | (dota['identity_hate']==1)]
offensive[offensive['toxic']==1].groupby('text').size().sort_values(ascending=False).head(20)

text
fuck you                         4494
bitch                            3699
fuck off                         3679
idiots                           2043
shut up                          1800
pussy                            1756
stupid                           1632
fucking                          1614
noob jug idiot dog ugly nigga    1052
BITCH                             927
FUCKING                           923
moron                             919
loser                             787
suck my dick                      777
fucking noob                      741
bitches                           695
asshole                           689
piece of shit                     565
mother fucker                     545
retarded                          528
dtype: int64

In [253]:
print(dota['toxic'].sum(), 'toxic', "messages out of", len(dota['toxic']), 
          "({}%)".format(round(dota['toxic'].sum()/len(dota['toxic']) * 100, 2)))

382083 toxic messages out of 6914913 (5.53%)


## Cleaning 'Obscene'

In [262]:
offensive[offensive['obscene']==1].groupby('text').size().sort_values(ascending=False).head(20)

text
fuck you                         4494
shit                             4300
bitch                            3699
fuck off                         3679
idiots                           2043
pussy                            1756
stupid                           1632
fucking                          1614
holy shit                        1101
noob jug idiot dog ugly nigga    1052
BITCH                             927
FUCKING                           923
moron                             919
suck my dick                      777
fucking noob                      741
bitches                           695
asshole                           689
the fuck                          656
piece of shit                     565
fuck this game                    560
dtype: int64

* Same top 20 as 'toxic'. Obscene seems to be classified well enough.

## Cleaning 'Threat'

In [264]:
offensive[offensive['threat']==1].groupby('text').size().sort_values(ascending=False).head(20)

text
kill him            751
kill yourself       200
kill me?            126
i kill you           88
go kill him          75
to kill me           69
you kill me          42
Kill him             42
dIE                  41
kill him             35
and kill             29
i will kill u        27
you cant kill me     27
go kill yourself     27
i will rape you      25
GO DIE               23
to kill              22
i will kill you      21
and kill me          21
kill?                20
dtype: int64

Many of the most common threatening texts are taken out of context and use the word "kill" as part of gameplay (not as threats). Some texts that contain that word are still threatening and will be kept labeled as 'threat'. Others will be re-labeled below.

In [283]:
dota.loc[dota['text'].str.lower()==("kill him"), ['toxic', 'threat']] = 0
dota.loc[dota['text'].str.lower()==("kill me?"), ['toxic', 'threat']] = 0
dota.loc[dota['text'].str.lower()==("i kill you"), ['toxic', 'threat']] = 0
dota.loc[dota['text'].str.lower()==("go kill him"), ['toxic', 'threat']] = 0
dota.loc[dota['text'].str.lower()==("to kill me"), ['toxic', 'threat']] = 0
dota.loc[dota['text'].str.lower()==("you kill me"), ['toxic', 'threat']] = 0
dota.loc[dota['text'].str.lower()==("die"), ['toxic', 'threat']] = 0
dota.loc[dota['text'].str.lower()==("kill him "), ['toxic', 'threat']] = 0
dota.loc[dota['text'].str.lower()==("and kill"), ['toxic', 'threat']] = 0
dota.loc[dota['text'].str.lower()==("you cant kill me"), ['toxic', 'threat']] = 0
dota.loc[dota['text'].str.lower()==("to kill"), ['toxic', 'threat']] = 0
dota.loc[dota['text'].str.lower()==("and kill me"), ['toxic', 'threat']] = 0
dota.loc[dota['text'].str.lower()==("kill?"), ['toxic', 'threat']] = 0

In [272]:
offensive = dota[(dota['toxic']==1) | (dota['threat']==1) | (dota['insult']==1) | (dota['identity_hate']==1)]
offensive[offensive['threat']==1].groupby('text').size().sort_values(ascending=False).head(20)

text
kill yourself       200
i will kill u        27
go kill yourself     27
i will rape you      25
GO DIE               23
i will kill you      21
kill you             20
over kill            19
my kill              18
JUST DIE             17
AND DIE              17
KILL AM              16
u kill me?           16
let me kill you      15
I DIE                15
ill kill you         14
Kill yourself        14
ill kill u           14
to kill me?          12
DONT KILL            12
dtype: int64

In [284]:
dota.loc[dota['text'].str.lower()==("kill you"), ['toxic', 'threat']] = 0
dota.loc[dota['text'].str.lower()==("over kill"), ['toxic', 'threat']] = 0
dota.loc[dota['text'].str.lower()==("my kill"), ['toxic', 'threat']] = 0
dota.loc[dota['text'].str.lower()==("just die"), ['toxic', 'threat']] = 0
dota.loc[dota['text'].str.lower()==("and die"), ['toxic', 'threat']] = 0
dota.loc[dota['text'].str.lower()==("kill am"), ['toxic', 'threat']] = 0
dota.loc[dota['text'].str.lower()==("u kill me?"), ['toxic', 'threat']] = 0
dota.loc[dota['text'].str.lower()==("let me kill you"), ['toxic', 'threat']] = 0
dota.loc[dota['text'].str.lower()==("i die"), ['toxic', 'threat']] = 0
dota.loc[dota['text'].str.lower()==("to kill me?"), ['toxic', 'threat']] = 0
dota.loc[dota['text'].str.lower()==("dont kill"), ['toxic', 'threat']] = 0

In [274]:
offensive = dota[(dota['toxic']==1) | (dota['threat']==1) | (dota['insult']==1) | (dota['identity_hate']==1)]
offensive[offensive['threat']==1].groupby('text').size().sort_values(ascending=False).head(20)

text
kill yourself       200
go kill yourself     27
i will kill u        27
i will rape you      25
GO DIE               23
i will kill you      21
ill kill u           14
ill kill you         14
Kill yourself        14
he kill me           12
|die                 12
i want die           12
to kill u            11
and u kill me        11
I will rape you      10
you kill me?         10
and kill him         10
FUCKING DIE          10
i'll kill u          10
kill m,e             10
dtype: int64

In [285]:
dota.loc[dota['text'].str.lower()==("he kill me"), ['toxic', 'threat']] = 0
dota.loc[dota['text'].str.lower()==("|die"), ['toxic', 'threat']] = 0
dota.loc[dota['text'].str.lower()==("i wont die"), ['toxic', 'threat']] = 0
dota.loc[dota['text'].str.lower()==("to kill u"), ['toxic', 'threat']] = 0
dota.loc[dota['text'].str.lower()==("and u kill me"), ['toxic', 'threat']] = 0
dota.loc[dota['text'].str.lower()==("you kill me?"), ['toxic', 'threat']] = 0
dota.loc[dota['text'].str.lower()==("and kill him"), ['toxic', 'threat']] = 0
dota.loc[dota['text'].str.lower()==("let me kill you"), ['toxic', 'threat']] = 0
dota.loc[dota['text'].str.lower()==("i'll kill u"), ['toxic', 'threat']] = 0
dota.loc[dota['text'].str.lower()==("kill m,e"), ['toxic', 'threat']] = 0

In [277]:
offensive = dota[(dota['toxic']==1) | (dota['threat']==1) | (dota['insult']==1) | (dota['identity_hate']==1)]
offensive[offensive['threat']==1].groupby('text').size().sort_values(ascending=False).head(20)

text
kill yourself       200
i will kill u        27
go kill yourself     27
i will rape you      25
GO DIE               23
i will kill you      21
ill kill you         14
Kill yourself        14
ill kill u           14
i want die           12
FUCKING DIE          10
I will rape you      10
i wont kill u        10
good kill            10
to kill you          10
you die?              9
and kill you          9
i can kill u          9
die.                  8
and kill u            8
dtype: int64

* The texts lower in the series have very small counts and will be left untouched.

In [279]:
print(dota['threat'].sum(), 'threat', "messages out of", len(dota['threat']), 
          "({}%)".format(round(dota['threat'].sum()/len(dota['threat']) * 100, 2)))

4222 threat messages out of 6914913 (0.06%)


## Cleaning 'Insult'

In [286]:
offensive[offensive['insult']==1].groupby('text').size().sort_values(ascending=False).head(20)

text
fuck you                         4494
shit                             4300
bitch                            3699
fuck off                         3679
idiots                           2043
stupid                           1632
fucking                          1614
holy shit                        1101
noob jug idiot dog ugly nigga    1052
BITCH                             927
FUCKING                           923
moron                             919
loser                             787
suck my dick                      777
fucking noob                      741
bitches                           695
asshole                           689
the fuck                          656
piece of shit                     565
fuck this game                    560
dtype: int64

Very similar to the most common toxic messages. The same re-labeled messages will be re-labeled for 'insult' too, along with other non-insulting swear words.

In [290]:
dota.loc[dota['text'].str.lower()==("shit"), 'insult'] = 0
dota.loc[dota['text'].str.lower()==("fucking"), 'insult'] = 0
dota.loc[dota['text'].str.lower()==("holy shit"), 'insult'] = 0
dota.loc[dota['text'].str.lower()==("the fuck"), 'insult'] = 0
dota.loc[dota['text'].str.lower()==("fuck this game"), 'insult'] = 0
dota.loc[dota['text'].str.lower()=="what the fuck", 'insult'] = 0

In [294]:
# Top 21-40 most insulting messages
offensive = dota[(dota['toxic']==1) | (dota['threat']==1) | (dota['insult']==1) | (dota['identity_hate']==1)]
offensive[offensive['insult']==1].groupby('text').size().sort_values(ascending=False)[21:41]

text
dick                424
STUPID              424
pathetic            413
fucking retard      412
fuck you            407
fucking trash       372
fuck this team      370
lucker              366
fucking noobs       366
run bitch           363
no shit             361
RETARDS             361
fuck this           357
eat shit            349
faggots             341
morons              340
holy fuck           332
FUCKING MORON       326
shut the fuck up    322
noob shit           317
dtype: int64

In [295]:
dota.loc[dota['text'].str.lower()=="no shit", 'insult'] = 0
dota.loc[dota['text'].str.lower()=="fuck this", 'insult'] = 0
dota.loc[dota['text'].str.lower()=="holy fuck", 'insult'] = 0

Because the purpose of ridding toxic players to make sure other players feel okay playing the game, 'offense' will be looked into deeper than the prior labels.
* 'Threat' is important too, but there are much less threatening messages, meaning there is also a smaller count of incorrectly classified threatening messages.

In [300]:
# Top 41-60 most insulting messages
offensive = dota[(dota['toxic']==1) | (dota['threat']==1) | (dota['insult']==1) | (dota['identity_hate']==1)]
offensive[offensive['insult']==1].groupby('text').size().sort_values(ascending=False)[41:61]

text
ez as fuck          266
fucking shit        264
get fucked          261
you are             257
so stupid           253
GAY                 253
dumb                248
dumbass             244
fuck it             242
this shit           240
sucks               239
idiot?              238
fuck you all        238
fucking retards     229
go fuck yourself    227
shit                220
Fuck off            219
fuck u all          218
fucking             213
as fuck             195
dtype: int64

In [301]:
dota.loc[dota['text'].str.lower()=="you are", ['toxic', 'insult']] = 0
dota.loc[dota['text'].str.lower()=="fuck it", ['toxic', 'insult']] = 0
dota.loc[dota['text'].str.lower()=="this shit", ['toxic', 'insult']] = 0
dota.loc[dota['text'].str.lower()=="shit", ['toxic', 'insult']] = 0
dota.loc[dota['text'].str.lower()=="fucking", ['toxic', 'insult']] = 0
dota.loc[dota['text'].str.lower()=="as fuck", ['toxic', 'insult']] = 0

In [303]:
# Top 61-80 most insulting messages
offensive = dota[(dota['toxic']==1) | (dota['threat']==1) | (dota['insult']==1) | (dota['identity_hate']==1)]
offensive[offensive['insult']==1].groupby('text').size().sort_values(ascending=False)[61:100]

text
fuck him                                                181
fuck my team                                            181
fucking russians                                        180
fucking idiots                                          177
Bitch                                                   173
retard?                                                 172
you                                                     171
ASSHOLE                                                 165
fuck team                                               164
useless shit                                            163
CUNT                                                    162
team retard                                             161
niggers                                                 160
BITCHES                                                 160
dickhead                                                159
stupid                                                  156
suck dick                          

In [304]:
dota.loc[dota['text'].str.lower()=="you", ['toxic', 'insult']] = 0
dota.loc[dota['text'].str.lower()=="oh fuck", ['toxic', 'insult']] = 0
dota.loc[dota['text'].str.lower()=="fuck that", ['toxic', 'insult']] = 0
dota.loc[dota['text'].str.lower()=="fuck my life", ['toxic', 'insult']] = 0

In [305]:
print(dota['insult'].sum(), 'insulting', "messages out of", len(dota['insult']), 
          "({}%)".format(round(dota['insult'].sum()/len(dota['insult']) * 100, 2)))

258246 insulting messages out of 6914913 (3.73%)


## Cleaning 'Identity_hate'

In [308]:
offensive = dota[(dota['toxic']==1) | (dota['threat']==1) | (dota['insult']==1) | (dota['identity_hate']==1)]
offensive[offensive['identity_hate']==1].groupby('text').size().sort_values(ascending=False).head(20)

text
noob jug idiot dog ugly nigga    1052
faggots                           341
GAY                               253
niggers                           160
Niggers tongue my anus.           134
Gay                               133
Tongue my anus, niggers.          122
really nigga                      108
really nigga?                      84
gay?                               82
rly nigga                          70
fucking faggot                     61
fucking gay                        57
skywrath hates niggers             56
gay shit                           56
thats gay                          44
he is gay                          41
u gay                              39
fucking nigger                     38
my nigga                           37
dtype: int64

* Identity_hate seems to be classified well enough already.

In [310]:
# dota.to_csv("dota_cleaned_labels.csv", index=False)